## Setting

In [75]:
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os
import pathlib
import requests
import schedule
import time

In [76]:
envpath = pathlib.Path('./.env')
load_dotenv(envpath)

True

## Main

In [77]:
def get_soup(url):
  """
  urlからBeautifulSoupオブジェクトを取得する
  
  Parameters
  ----------
  url : str
    対象のURL
      
  Returns
  -------
  soup : BeautifulSoup
    BeautifulSoupオブジェクト
  """
  time.sleep(1)
  res = requests.get(url)
  return BeautifulSoup(res.text, 'html.parser')

In [78]:
def nintendo_stock_checker(url):
  """
  urlから任天堂スイッチの在庫情報を取得する
  
  Parameters
  ----------
  url : str
    対象のURL
      
  Returns
  -------
  bool : bool
    在庫があるかどうか
  """
  soup = get_soup(url)
  item_name_elements = soup.select('.c-productList--item__name')
  for item_name_element in item_name_elements:
    if 'Nintendo Switch Proコントローラー' in item_name_element.text:
      return True
  return False

In [79]:
def send_line_notify(notification_message):
    """
    LINEに通知する

    環境変数 YOUR_LINE_NOTIFY_TOKEN にトークンを設定しておくこと

    Parameters
    ----------
    notification_message : str
        通知するメッセージ

    Returns
    -------
    None
    """
    line_notify_token = os.environ.get('YOUR_NOTIFY_TOKEN')
    line_notify_api = 'https://notify-api.line.me/api/notify'
    headers = {'Authorization': f'Bearer {line_notify_token}'}
    data = {'message': f'message: {notification_message}'}
    requests.post(line_notify_api, headers = headers, data = data)

In [80]:
def main():
    global flag
    url = 'https://store-jp.nintendo.com/list/hardware-accessory/controller/?prefn1=hasInventory&prefv1=true'
    send_text = f'在庫あり\nurl: {url}'
    if nintendo_stock_checker(url):
        send_line_notify(send_text)
        flag = True

In [82]:
schedule.clear()
schedule.every(3).minutes.do(main)

Every 3 minutes do main() (last run: [never], next run: 2022-09-18 22:50:55)

In [ ]:
flag = False
while True:
  if flag:
    break
  schedule.run_pending()
  time.sleep(50)

hoge
hoge


KeyboardInterrupt: 